In [1]:
pip install datasetsforecast

In [2]:
pip install statsforecast

In [3]:
pip install mfles

In [4]:
import pandas as pd

Y_df_m5 = pd.read_parquet('https://m5-benchmarks.s3.amazonaws.com/data/train/target.parquet')

Y_df_m5 = Y_df_m5.rename(columns={
    'item_id': 'unique_id',
    'timestamp': 'ds',
    'demand': 'y'
})

Y_df_m5.head()
Y_df_m5['unique_id'] = Y_df_m5['unique_id'].astype(str)


In [5]:
Y_df_m5 = Y_df_m5.sort_values(['ds'],ascending=False)
Y_df_m5['count'] = Y_df_m5.groupby('unique_id').cumcount()

In [6]:
import pandas as pd

from MFLES.Forecaster import optimize_from_df

configs = { #params to try out to optimize for
    'smoother': [True, False], #test both ETS and SMA
    'ma': [7], #smooth out one seasonal period when using moving average
    'seasonality_weights': [True],
    'seasonal_period': [None, [7], [7, 30, 365]], #try both types of possible seasonality
    }

short_df = Y_df_m5[Y_df_m5['unique_id'].isin(Y_df_m5['unique_id'].unique()[:5])] #did this for testing
output = optimize_from_df(Y_df_m5,
                          forecast_horizon=28,
                          test_size=14,
                          n_steps=1, #don't do rolling testing to save time / rolling should make the optimization better
                          metric='mse',
                          params=configs,
                          verbose=False,
                          seasonal_period=[7, 30],
                          freq='D')

output = output.fillna('forecast_set')
output = output[output['y'] == 'forecast_set']

/usr/local/lib/python3.10/dist-packages/MFLES/Model.py:41: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def lasso_nb(X, y, alpha, tol=0.001, maxiter=10000):
/usr/local/lib/python3.10/dist-packages/MFLES/Model.py:164: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def median(y, seasonal_period):
/usr/local/lib/python3.10/dist-packages/MFLES/Model.py:180: NumbaDep

In [7]:
output = output.reset_index(drop=True).set_index('unique_id')

In [8]:
from datasetsforecast.m5 import M5, M5Evaluation

In [9]:
from datasetsforecast.m5 import M5, M5Evaluation
from statsforecast import StatsForecast

### Evaluator
def evaluate_forecasts(df, model, model_name):
    Y_hat = df.set_index('ds', append=True)[model].unstack()
    *_, S_df = M5.load('data')
    Y_hat = S_df.merge(Y_hat, how='left', on=['unique_id'])
    eval_ = M5Evaluation.evaluate(y_hat=Y_hat, directory='./data')
    eval_ = eval_.rename(columns={'wrmsse': f'{model_name}_{model}_wrmsse'})
    return eval_

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [10]:
m5_eval_df = pd.concat([
    evaluate_forecasts(output[['mfles', 'ds']], 'mfles', 'MFLES'),
], axis=1)
m5_eval_df.T

,Total,Level1,Level2,Level3,Level4,Level5,Level6,Level7,Level8,Level9,Level10,Level11,Level12
MFLES_mfles_wrmsse,0.643943,0.365268,0.474021,0.543244,0.41759,0.498432,0.545146,0.615165,0.614845,0.701979,1.024448,0.986423,0.940751


In [11]:
print(m5_eval_df.T)

                       Total    Level1    Level2    Level3   Level4    Level5  \
MFLES_mfles_wrmsse  0.643943  0.365268  0.474021  0.543244  0.41759  0.498432   

                      Level6    Level7    Level8    Level9   Level10  \
MFLES_mfles_wrmsse  0.545146  0.615165  0.614845  0.701979  1.024448   

                     Level11   Level12  
MFLES_mfles_wrmsse  0.986423  0.940751  
